# QIR generation with Q\#

_Tutorials are narrated examples of using QIR as a part of some workflow or with a particular set of tools._

- using the `%qir` magic command in notebooks: https://docs.microsoft.com/en-us/qsharp/api/iqsharp-magic/qir
- use the project file flag to specify QIR compilation of a Q# program: https://github.com/microsoft/qsharp-compiler/tree/main/src/QsCompiler/QirGeneration